In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ROOT import TFile, TChain, TTree, TCanvas
from PMTAnalysisTools import PMT, Ophit, Flash, Event

Welcome to JupyROOT 6.22/08


In [2]:
mj_filepath = "/pnfs/icarus/scratch/users/ascarpel/trigger/6105/ana/" #for majority trigger
mb_filepath = "/pnfs/icarus/scratch/users/ascarpel/trigger/6031/ana/" #for minbias

mj_geotree                = TChain("flashana/geotree")
mj_eventstree             = TChain("flashana/eventstree")
mj_opflashCryoE_flashtree = TChain("flashana/opflashCryoE_flashtree/")
mj_opflashCryoE_ophittree = TChain("flashana/opflashCryoE_ophittree")
mj_opflashCryoW_flashtree = TChain("flashana/opflashCryoW_flashtree")
mj_opflashCryoW_ophittree = TChain("flashana/opflashCryoW_ophittree")

mb_geotree                = TChain("flashana/geotree")
mb_eventstree             = TChain("flashana/eventstree")
mb_opflashCryoE_flashtree = TChain("flashana/opflashCryoE_flashtree")
mb_opflashCryoE_ophittree = TChain("flashana/opflashCryoE_ophittree")
mb_opflashCryoW_flashtree = TChain("flashana/opflashCryoW_flashtree")
mb_opflashCryoW_ophittree = TChain("flashana/opflashCryoW_ophittree")

mj_filenames = []
with open(mj_filepath+"filesana.list") as f:
    mj_filenames  = f.readlines()
mj_filenames = [x.strip() for x in mj_filenames] 

mb_filenames = []
with open(mb_filepath+"filesana.list") as f:
    mb_filenames = f.readlines()

for mj_filename in mj_filenames:
    mj_geotree.Add(mj_filename)
    mj_eventstree.Add(mj_filename)
    mj_opflashCryoE_flashtree.Add(mj_filename)
    mj_opflashCryoE_ophittree.Add(mj_filename)
    mj_opflashCryoW_flashtree.Add(mj_filename)
    mj_opflashCryoW_ophittree.Add(mj_filename)
    break # for testing purposes

for mb_filename in mb_filenames:
    mb_geotree.Add(mj_filename)
    mb_eventstree.Add(mj_filename)
    mb_opflashCryoE_flashtree.Add(mj_filename)
    mb_opflashCryoE_ophittree.Add(mj_filename)
    mb_opflashCryoW_flashtree.Add(mj_filename)
    mb_opflashCryoW_ophittree.Add(mj_filename)
    break # for testing purposes


In [4]:
mb_pmts    = {}
mb_events  = {}

for record in mb_geotree:
    for channel in range(len(record.pmt_x)):
        mb_pmts[channel+1] = PMT((record.pmt_x,
                                  record.pmt_y,
                                  record.pmt_z))

for event in mb_eventstree:
    mb_events[event.event]=Event([event.run,
                                  event.event,
                                  event.timestamp,
                                  event.nflashes,
                                  event.nophits,
                                  event.beam_gate_start,
                                  event.beam_gate_width,
                                  event.beam_type])
    
for flash in mb_opflashCryoE_flashtree:
    _flash = Flash([flash.run,
                    flash.event,
                    flash.timestamp,
                    flash.flash_id,
                    flash.multiplicity,
                    flash.multiplicity_right,
                    flash.multiplicity_left,
                    flash.sum_pe,
                    flash.sum_pe_right,
                    flash.sum_pe_left,
                    flash.flash_time,
                    flash.flash_y,
                    flash.flash_width_y,
                    flash.flash_z,
                    flash.flash_width_z])
    mb_events[flash.event].AddFlash(_flash)


for flash in mb_opflashCryoW_flashtree:
    _flash = Flash([flash.run,
                    flash.event,
                    flash.timestamp,
                    flash.flash_id,
                    flash.multiplicity,
                    flash.multiplicity_right,
                    flash.multiplicity_left,
                    flash.sum_pe,
                    flash.sum_pe_right,
                    flash.sum_pe_left,
                    flash.flash_time,
                    flash.flash_y,
                    flash.flash_width_y,
                    flash.flash_z,
                    flash.flash_width_z])
    mb_events[flash.event].AddFlash(_flash)
    
for ophit in mb_opflashCryoE_ophittree:
    
    _ophit = Ophit([ophit.run,
                    ophit.event,
                    ophit.timestamp,
                    ophit.flash_id,
                    ophit.channel_id,
                    ophit.integral,
                    ophit.amplitude,
                    ophit.start_time,
                    ophit.abs_start_time,
                    ophit.pe,
                    ophit.width,
                    ophit.fast_to_total])
    
    #print(ophit.event)
    #((mb_events[_ophit.event]).GetFlashes())[ophit.flash_id] = _ophit
    
    #print(_ophit.event)
    (((mb_events[_ophit.event]).GetFlashes())[_ophit.flash_id]).AddOpHit(_ophit)

for ophit in mb_opflashCryoW_ophittree:
    
    _ophit = Ophit([ophit.run,
                    ophit.event,
                    ophit.timestamp,
                    ophit.flash_id,
                    ophit.channel_id,
                    ophit.integral,
                    ophit.amplitude,
                    ophit.start_time,
                    ophit.abs_start_time,
                    ophit.pe,
                    ophit.width,
                    ophit.fast_to_total])